In [1]:
import os
import sys
import pandas as pd
from tabulate import tabulate

(x_train,       # 00. wzorzec (wektor dł 256) np. array([10, 10, 52, 48, ...
    y_train,        # 01. etykieta, np. 'fsbndm'
    x_test,         # 02. jak x_train ale dla testow
    y_test,         # 03. jak y_train ale dla testow
    X_train_time,   # 04. DataFrame ze wszystkimi danymi dla kazdego wzorca
    X_test_time,    # 05. j.w. dla testowego
    Y_train_time,   # 06. chyba to samo co X_train_time
    Y_test_time,    # 07. chyba to samo co X_test_time
    y_train_onehot, # 08. one hot dla train
    y_test_onehot,  # 09. one hot dla test
    y_test_time,    # 10. czasy dla kazdego z testowanych algorytmow (w chwili pisania 27)
    y_train_time,   # 11. czasy dla kazdego z testowanych algorytmow (w chwili pisania 27)
    X_columns,      # 12. nazwy kolumn z kolejnymi symbolami wzorca np. pattern_0 (pierwszy znak), itd.
    alg_name_to_id, # 13. mapowanie nazwy algorytmu na jego ID
    algorithms,     # 14. Nazwy wszystkich algorytmow ktore zostaly przetestowane (czy moze wygraly przynajmniej raz?)
    algorithm_to_onehot
) = pd.read_pickle('dataset-full.pickle')

In [2]:
# 3 najlepsze algorytmy w kategorii dataset, m
df = Y_test_time

tmp = df [['dataset', 'm', 'algorithm', 'pattern']].groupby(
    ['dataset', 'm', 'algorithm']
).count().rename(columns={'pattern': '#'}).reset_index()

n_best = 3 # czyli 3

records = []
for idx, gr in tmp.sort_values(by=['dataset', 'm', '#'], ascending=False).groupby(['dataset', 'm']):
    dataset, m = idx
    qwe = []
    coln = 0
    summ = sum(gr['#'])
    for idx2, el in gr.iterrows():
        _, _, alg, wins = el.values
        qwe += [[dataset, m, coln, f'{100*wins/summ} ({alg})']]
        coln += 1
        # qwe += [[dataset, m, coln, f'({alg})']]
        # coln += 1
    qwe = qwe[:n_best]
    if len(qwe) < n_best:
        for i in range(len(qwe), n_best):
            qwe += [[dataset, m, coln, '']]
            coln += 1
            # qwe += [[dataset, m, coln, '']]
            # coln += 1
    records.extend(qwe)
records

df = pd.DataFrame(records)
df = df.rename(columns={0:'dataset', 1:'m', 2:'n-th best', 3:'result'})
df = df[df['m'].isin([6, 8, 16, 32, 64, 128, 256])]
df = df.pivot(index=['dataset', 'n-th best'], columns='m').reset_index()
df.to_latex('table3.tex', index=False)
df

dataset n-th best         result                 \
m                                                      6              8   
0   data/dblp.xml/dblp.xml.50MB         0    70.0 (ebom)    62.5 (ebom)   
1   data/dblp.xml/dblp.xml.50MB         1  27.0 (fsbndm)  30.5 (fsbndm)   
2   data/dblp.xml/dblp.xml.50MB         2    3.0 (sebom)    7.0 (sebom)   
3             data/dna/dna.50MB         0      98.5 (so)     100.0 (so)   
4             data/dna/dna.50MB         1     1.5 (ebom)                  
5             data/dna/dna.50MB         2                                 
6     data/english/english.50MB         0    86.0 (ebom)    88.0 (ebom)   
7     data/english/english.50MB         1  14.0 (fsbndm)  12.0 (fsbndm)   
8     data/english/english.50MB         2                                 
9     data/pitches/pitches.50MB         0    61.5 (ebom)    59.5 (ebom)   
10    data/pitches/pitches.50MB         1  23.0 (fsbndm)  23.5 (fsbndm)   
11    data/pitches/pitches.50MB         2   15.5 (sebom)   17.0 (sebom)   
12  data/proteins/proteins.50MB         0    70.0 (ebom)   51.5 (sebom)   
13  data/proteins/proteins.50MB         1   24.5 (sebom)    44.5 (ebom)   
14  data/proteins/proteins.50MB         2   5.5 (fsbndm)   4.0 (fsbndm)   
15    data/sources/sources.50MB         0    65.0 (ebom)    62.5 (ebom)   
16    data/sources/sources.50MB         1  35.0 (fsbndm)  37.5 (fsbndm)   
17    data/sources/sources.50MB         2                                 

                                                                               
m              16             32             64            128            256  
0   46.0 (fsbndm)  51.0 (fsbndm)      39.5 (br)   30.0 (sebom)   41.5 (askip)  
1     35.0 (ebom)   17.0 (sebom)  24.0 (fsbndm)      22.5 (br)  29.5 (fsbndm)  
2    19.0 (sebom)   16.5 (sbndm)    19.5 (ebom)  20.5 (fsbndm)   21.5 (sebom)  
3    93.0 (askip)   99.5 (askip)  100.0 (askip)  100.0 (askip)   45.5 (sfbom)  
4     4.0 (smith)     0.5 (ebom)                                 40.0 (askip)  
5      2.5 (ebom)                                                  8.5 (ebom)  
6     80.5 (ebom)    73.5 (ebom)    91.5 (ebom)    95.5 (ebom)    96.0 (ebom)  
7   19.5 (fsbndm)  24.5 (fsbndm)   6.5 (fsbndm)   2.5 (fsbndm)    3.5 (sebom)  
8                       1.0 (br)    1.0 (sebom)    1.0 (askip)   0.5 (fsbndm)  
9     41.0 (ebom)    32.5 (ebom)   43.5 (sebom)    46.0 (ebom)    50.0 (ebom)  
10   38.5 (sebom)   27.5 (sebom)    39.5 (ebom)   44.0 (sebom)   45.0 (sebom)  
11  20.5 (fsbndm)  21.5 (fsbndm)  10.0 (fsbndm)       6.5 (rf)       3.0 (rf)  
12   76.0 (sebom)   93.5 (sebom)   93.5 (sebom)   83.0 (sebom)   86.5 (sebom)  
13    23.5 (ebom)     5.5 (ebom)     6.5 (ebom)    17.0 (ebom)    13.5 (ebom)  
14   0.5 (fsbndm)   1.0 (fsbndm)                                               
15    71.5 (ebom)  48.5 (fsbndm)    82.0 (ebom)    95.0 (ebom)    93.5 (ebom)  
16  28.0 (fsbndm)    43.5 (ebom)   11.0 (sebom)    4.5 (sebom)    4.5 (sebom)  
17    0.5 (sebom)    5.0 (sebom)   7.0 (fsbndm)    0.5 (askip)    2.0 (askip)

In [3]:
# 3 najlepsze algorytmy w kategorii dataset
df = Y_test_time

tmp = df [['dataset', 'algorithm', 'pattern']].groupby(
    ['dataset', 'algorithm']
).count().rename(columns={'pattern': '#'}).reset_index()

records = []
for idx, gr in tmp.sort_values(by=['dataset', '#'], ascending=False).groupby(['dataset']):
    dataset = idx
    qwe = []
    coln = 0
    for idx2, el in gr.iterrows():
        _, alg, wins = el.values
        qwe += [[dataset, coln, f'{round(wins/60, 2)}% ({alg})']]
        coln += 1
    qwe = qwe[:3]
    if len(qwe) < 3:
        for i in range(len(qwe), 3):
            qwe += [[dataset, coln, '']]
            coln += 1
    records.extend(qwe)
records


df = pd.DataFrame(records)
df = df.rename(columns={0:'dataset', 1:'n-th best', 2:'result'})
df = df.pivot(index=['dataset'], columns='n-th best').reset_index()
print(df.to_markdown(index=False))

| ('dataset', '')                  | ('result', 0)   | ('result', 1)   | ('result', 2)   |
|:---------------------------------|:----------------|:----------------|:----------------|
| ('data/dblp.xml/dblp.xml.50MB',) | 15.02% (fsbndm) | 13.9% (ebom)    | 6.23% (sebom)   |
| ('data/dna/dna.50MB',)           | 21.58% (askip)  | 12.62% (so)     | 1.92% (ebom)    |
| ('data/english/english.50MB',)   | 33.55% (ebom)   | 6.12% (fsbndm)  | 0.22% (sebom)   |
| ('data/pitches/pitches.50MB',)   | 19.48% (ebom)   | 11.63% (sebom)  | 7.42% (fsbndm)  |
| ('data/proteins/proteins.50MB',) | 26.9% (sebom)   | 12.28% (ebom)   | 0.82% (fsbndm)  |
| ('data/sources/sources.50MB',)   | 27.37% (ebom)   | 11.52% (fsbndm) | 0.9% (sebom)    |


In [4]:
# 3 najlepsze algorytmy w kategorii m
df = Y_test_time

tmp = df [['m', 'algorithm', 'pattern']].groupby(
    ['m', 'algorithm']
).count().rename(columns={'pattern': '#'}).reset_index()


records = []
for idx, gr in tmp.sort_values(by=['m', '#'], ascending=False).groupby(['m']):
    dataset = idx
    qwe = []
    coln = 0
    for idx2, el in gr.iterrows():
        _, alg, wins = el.values
        qwe += [[dataset, coln, f'{round(wins/60, 2)}% ({alg})']]
        coln += 1
    qwe = qwe[:3]
    if len(qwe) < 3:
        for i in range(len(qwe), 3):
            qwe += [[dataset, coln, '']]
            coln += 1
    records.extend(qwe)
records


df = pd.DataFrame(records)
df = df.rename(columns={0:'m', 1:'n-th best', 2:'result'})
df = df.pivot(index=['m'], columns='n-th best').reset_index()
print(df.to_markdown(index=False))

| ('m', '')   | ('result', 0)   | ('result', 1)   | ('result', 2)   |
|:------------|:----------------|:----------------|:----------------|
| (4,)        | 11.3% (ebom)    | 6.72% (fsbndm)  | 1.2% (so)       |
| (6,)        | 11.8% (ebom)    | 3.48% (fsbndm)  | 3.28% (so)      |
| (8,)        | 10.57% (ebom)   | 3.58% (fsbndm)  | 3.33% (so)      |
| (10,)       | 10.0% (ebom)    | 3.48% (fsbndm)  | 3.32% (sebom)   |
| (12,)       | 9.43% (ebom)    | 4.03% (sebom)   | 3.4% (fsbndm)   |
| (14,)       | 9.0% (ebom)     | 4.32% (sebom)   | 3.72% (fsbndm)  |
| (16,)       | 8.47% (ebom)    | 4.47% (sebom)   | 3.82% (fsbndm)  |
| (24,)       | 6.67% (ebom)    | 5.22% (fsbndm)  | 4.48% (sebom)   |
| (32,)       | 5.53% (ebom)    | 4.88% (fsbndm)  | 4.78% (sebom)   |
| (64,)       | 7.97% (ebom)    | 5.48% (sebom)   | 3.43% (askip)   |
| (128,)      | 8.98% (ebom)    | 5.42% (sebom)   | 3.63% (askip)   |
| (256,)      | 8.78% (ebom)    | 5.37% (sebom)   | 2.83% (askip)   |
